# Caso Práctico: Integración del ChatBot con servicios externos

<div style="background-color:#D9EEFF;color:black;padding:2%;">
<h2>Enunciado del caso práctico</h2>

En este caso práctico, se propone al alumno la integración del ChatBot implementando en el caso práctico anterior con el servicio de correo electrónico de Gmail.

La aplicación debe ser capaz de realizar diferentes funciones sobre los correos electrónicos del usuario. Por ejemplo, debe ser capaz de buscar todos los correos electrónicos de un remitente concreto y resumir su contenido.

</div>

# Resolución del caso práctico

## 0. Instalación de librerías externas

In [ ]:
#!pip install openai==0.28

## 1. Lectura de la API Key

In [ ]:
path_pc_repo = "C:/Users/Osvaldo/OneDrive/Documents/3. Projectos Visual Studio/GenerativeAI/"
path_repo = "data/data"
key_path = "/5. Aplicaciones LLMs-ChatBot/"
file_info = "/api-key.txt"

In [ ]:
import openai

path_file = path_pc_repo + key_path + file_info

#with open("/content/drive/MyDrive/api-keys/secret-key.txt") as f:
#  openai.api_key = f.readline()

with open(path_file ) as f:
  openai.api_key = f.readline()

## 2. Selección del modelo

En la función `obtener_completion` que habíamos implementado para casos prácticos anteriores, únicamente hacíamos uso del `role` de `user`:

```python
def obtener_completion(prompt, model="gpt-3.5-turbo"):
  mensaje = [{"role": "user", "content": prompt}]
  respuesta = openai.ChatCompletion.create(
      model=model,
      messages=mensaje,
      temperature=0, # Este hiperparámetro controla la aleatoriedad del modelo
  )
  return respuesta.choices[0].message["content"]
```

En este caso práctico, vamos a tener que explorar el [resto de roles que nos proporcionar OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api) para sus LLMs y la función que tienen cada uno de ellos:


*  Rol `user`: Representa al usuario final que interactúa con el LLM a través de un chat.
*  Rol `assistant`: Representa al LLM que estemos utilizando, en este caso, `gpt-3.5-turbo`.
*  Rol `system`: Este rol representa al desarrollador de sistema. Permite proporcionar instrucciones "root" al LLM para que se sigan durante la conversación con el usuario.

In [ ]:
def obtener_completion(mensajes, model="gpt-3.5-turbo"):
  respuesta = openai.ChatCompletion.create(
      model=model,
      messages=mensajes,
      temperature=0, # Este hiperparámetro controla la aleatoriedad del modelo
  )
  return respuesta.choices[0].message["content"]

## 3. Integración con Gmail

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from __future__ import print_function

import os.path
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


def obtener_correos(remitente):
    """Proporciona los correos electrónicos del usuario."""
    creds = None
    if os.path.exists('/content/drive/MyDrive/api-keys/token.json'):
        creds = Credentials.from_authorized_user_file('/content/drive/MyDrive/api-keys/token.json', SCOPES)

    try:
        correos = """"""
        service = build('gmail', 'v1', credentials=creds)
        results = service.users().messages().list(userId='me', labelIds=['INBOX']).execute()
        # Recorremos todos los correos electrónicos
        for msg in results['messages']:
          mensaje = service.users().messages().get(userId='me', id=msg['id']).execute()
          # Comparamos el remitente del correo
          if remitente in str(mensaje['payload']['headers'][14]):
            # Extraemos el contenido del correo
            datos = base64.b64decode(mensaje['payload']['parts'][0]['body']['data'], '-_')
            correos += f"""
'''
Contenido: {datos}
'''
"""
        return correos

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f'An error occurred: {error}')

In [ ]:
print(obtener_correos("shramos@protonmail.com"))

## 4. Implementación de un ChatBot interactivo

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = obtener_completion(context)
    try:
      remitente = eval(response)["remitente"]
      accion_solicitada = eval(response)["accion_solicitada"]
      correos = obtener_correos(remitente)
      context.pop(0) # Elimino del contexto la regla inicial
      context.append({'role': 'user', 'content': f"Realiza la siguiente acción: {accion_solicitada} sobre los siguientes correos electrónicos: {correos}"})
      response = obtener_completion(context)
    except:
      pass
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)

In [ ]:
def end_chat(event):
    panels.append(pn.pane.Alert("Chat terminado por el usuario.", alert_type='success'))
    context.append({'role': 'system', 'content':"Despídete del usuario de manera amable y amigable."})
    response = obtener_completion(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = []

context = [ {'role':'system', 'content':
"""
Debes solicitar al usuario que te indique una dirección de correo electrónico que no sea la suya y la tarea que quiere realizar sobre sus correos electrónicos. \
Con esta información, debes generar un JSON con las siguientes claves. No debes generar nada más.
"remitente":<correo electrónico indicado por el usuario>
"accion_solicitada":<descripción de la accion solicitada por el usuario sobre los correos electrónicos del remitente>
"""} ]


inp = pn.widgets.TextInput(value="Hola", placeholder='Introduce texto aqui...')
button_conversation = pn.widgets.Button(name="Chat!")
button_end_chat = pn.widgets.Button(name="Terminar Chat")

button_end_chat.on_click(end_chat)

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation, button_end_chat),
    pn.panel(interactive_conversation, loading_indicator=True, sizing_mode="stretch_both"),
)

dashboard